In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json
from secret_data import userpass 
import pandas as pd 
import re 

In [ ]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import json
import re

# Load the CSV
odp = pd.read_csv('Socrata Imap Link Replacements.csv')

# Credentials
domain = "opendata.maryland.gov"
username = userpass[0]
password = userpass[1]

for index, row in odp.iterrows():
    dataset_id = row['UID']
    metadata_url = f"https://{domain}/api/views/{dataset_id}"
    
    # Make the request
    response = requests.get(metadata_url, auth=HTTPBasicAuth(username, password))
    
    # Check the response
    if response.status_code == 200:
        metadata = response.json()
        print("Metadata retrieved successfully!")
        #print(metadata)
    else:
        print(f"Failed to retrieve metadata. Status code: {response.status_code}")
        print(response.text)

    # Check if 'new link' is not null and not equal to "#NA"
    if pd.notna(row['new link']) and row['new link'] != "#NA" and row['new link']!="0/2" and row ['new link']!="0/0"and row ['new link']!="0/1":
        metadata['attributionLink'] = row['new link']
        
        metadata['description'] = re.sub(
        r"((?:Map Service Link:|Service Link:|Service Layer Link:)\s*)http[s]?://\S+",
        lambda m: m.group(1) + row['found link'],
        metadata['description'])
        print(dataset_id)
        # Send the updated metadata back to the API
        update_response = requests.put(
            metadata_url,
            auth=HTTPBasicAuth(username, password),
            headers={"Content-Type": "application/json"},
            data=json.dumps(metadata)
        )

        if update_response.status_code in [200, 202]:
            print("Source link updated successfully!")
        else:
            raise Exception(f"Failed to update metadata: {update_response.status_code}, {update_response.text}, {dataset_id}")
    else:
        print('No matching value')